### Image Dataset - `Butterflies, Moths, and Skippers`

In this notebook we are going to scrape the data from [this site](https://www.insectimages.org/index.cfm) for an image classification dataset.

### Innstallation

First we need to install `selenium` if not installed.

In [4]:
# pip install selenium -q

Note: you may need to restart the kernel to use updated packages.


### Imports
Next we are going to import all the packages that we will use in this notebook for this task.

In [57]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import tqdm
import random
import shutil

from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

Next we are going to set the seed for random operations for reproducivity.

In [58]:
SEED = 23

random.seed(SEED)

Next we are going to create an instance of a Chrome driver

In [105]:
driver = webdriver.Chrome()
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="130c802a10357cde800f972a1ed01e32")>

We are then going to define the path to where we want to get the images of the insects from.

In [106]:
urls_to_images ='https://www.insectimages.org/browse/taxthumb.cfm?order=131'

In [107]:
driver.get(urls_to_images)

From the website the images are paginated so we need to do the infinite scrolling So the function:

```py
def scroll_down(driver, scroll_pause_time=1.0):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
scroll_down(driver, 20)
```
Is used to do the inifite automatic scrolling of the page using `selenium`.


The following function will scrape a single page and returns us a list of dictionaries with `image` and `class` of the insect.

In [108]:
def scrape_page(soup):
    res = list()
    img_container = soup.find('div', {'id': 'imagecontainer'})
    for row in img_container.find_all('div', {'class': 'row vertical'}):
        for item in row.find_all('div', {'class': 'pointer item text-center well'}):
            try:
                img = f"https:{item.find('img')['src']}"
                class_ = item.find('div', {'class': 'img-foot'}).contents[0].split(' ')[-1].strip().lower()
                res.append({'class': class_, 'image': img})
            except Exception:
                pass
    return res
    

We are going to define the number of pages we want to scroll  and start doing webscraping to get the images and their class names.

In [110]:
number_of_pages = 900
data = []
for page in range(number_of_pages):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    res = scrape_page(soup)
    data.extend(res)
    print(f"Got {len(res)} for page {page+1}...")


Got 186 for page 1...
Got 210 for page 2...
Got 234 for page 3...
Got 258 for page 4...
Got 281 for page 5...
Got 301 for page 6...
Got 322 for page 7...
Got 322 for page 8...
Got 346 for page 9...
Got 370 for page 10...
Got 394 for page 11...
Got 418 for page 12...
Got 442 for page 13...
Got 466 for page 14...
Got 489 for page 15...
Got 513 for page 16...
Got 536 for page 17...
Got 559 for page 18...
Got 583 for page 19...
Got 607 for page 20...
Got 631 for page 21...
Got 655 for page 22...
Got 678 for page 23...
Got 701 for page 24...
Got 724 for page 25...
Got 747 for page 26...
Got 766 for page 27...
Got 789 for page 28...
Got 811 for page 29...
Got 835 for page 30...
Got 858 for page 31...
Got 882 for page 32...
Got 903 for page 33...
Got 926 for page 34...
Got 946 for page 35...
Got 963 for page 36...
Got 987 for page 37...
Got 1011 for page 38...
Got 1032 for page 39...
Got 1053 for page 40...
Got 1074 for page 41...
Got 1098 for page 42...
Got 1122 for page 43...
Got 1146 for p

KeyboardInterrupt: 

We can check the first `2` examples in the scrapped data.

In [111]:
data[:2]

[{'class': 'butterfly',
  'image': 'https://bugwoodcloud.org/images/384x256/9009084.jpg'},
 {'class': 'butterfly',
  'image': 'https://bugwoodcloud.org/images/384x256/9009083.jpg'}]

In [112]:
len(data)

2868852

In the following code cell we are going to create a dataframe based on the data that we have scrapped.

In [113]:
paired = [list(r.values()) for r in data]

df = pd.DataFrame(paired, columns=['class', 'image'])
df.head()

,class,image
0,butterfly,https://bugwoodcloud.org/images/384x256/900908...
1,butterfly,https://bugwoodcloud.org/images/384x256/900908...
2,moths,https://bugwoodcloud.org/images/384x256/900907...
3,moths,https://bugwoodcloud.org/images/384x256/900907...
4,moths,https://bugwoodcloud.org/images/384x256/900907...


The following function will rename `plural`to `singular` for class names. 

In [115]:
df['class'].value_counts()[:10]

class
moth           678438
webworm        256112
moth)          164326
caterpillar    138876
borer          113022
butterfly      102138
swallowtail    100645
budworm         79697
leafroller      72312
armyworm        59172
Name: count, dtype: int64

In [124]:
def rename_class(class_):
    obj = {
        "butterflies": "butterfly",
        "moths": "moth",
        "moth)": "moth",
        "hawk-moth": "moth",
        "silkmoth": "moth",
        "silkmoth": "hawkmoth",
        "buttferly": "butterfly",
          "skippers": "skipper", 
    "checkered-skipper": "skipper", 
    }
    try:
        return obj[class_]
    except KeyError:
        return class_
df['class'] = df['class'].apply(rename_class)

Next we are going to drop the duplicates columns based on the `image` column and filter them to `3` labels for insects which is:

1. `butterfly`
2. `moth`
3. `skipper`

In [128]:
df_unique = df.drop_duplicates(subset='image')
df_filtered = df_unique[df_unique['class'].isin(['butterfly', 'moth', "skipper"])]
df_filtered.head()

,class,image
0,butterfly,https://bugwoodcloud.org/images/384x256/900908...
1,butterfly,https://bugwoodcloud.org/images/384x256/900908...
2,moth,https://bugwoodcloud.org/images/384x256/900907...
3,moth,https://bugwoodcloud.org/images/384x256/900907...
4,moth,https://bugwoodcloud.org/images/384x256/900907...


In [142]:
print(df_filtered['class'].unique())

['butterfly' 'moth' 'skipper']


In [143]:
df_filtered.to_csv('pickle.csv', index=False)
print("Saved")

Saved


In [144]:
df_filtered = pd.read_csv('pickle.csv')

In [145]:
df_filtered['class'].value_counts()

class
moth         3569
butterfly     356
skipper       175
Name: count, dtype: int64

Next we are going to download these images into their respective directories based on their class names. Before we donload them we want to make sure that they are balanced.

In [146]:
min_size = df_filtered['class'].value_counts().min()
df_balanced = df_filtered.groupby('class').apply(lambda x: x.sample(min_size)).reset_index(drop=True)
df_balanced['class'].value_counts()

class
butterfly    175
moth         175
skipper      175
Name: count, dtype: int64

In [147]:
base_dir = "dataset"

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

for class_ in df_filtered['class'].unique():
    class_dir = os.path.join(base_dir, class_)
    if not os.path.exists(class_dir):
        os.mkdir(class_dir)


We are going to use the `ThreadPoolExecutor` from `concurrent.futures` to do multi-processing in downloading and saving the images concurrently. First let's check the number of `cpu's` that are in this computer.

In [148]:
num_workers = multiprocessing.cpu_count()
print("CPUs: {}".format(num_workers))

CPUs: 12


We can try to get the images and their classes from a pandas dataframe to a python list of tuples.

In [149]:
img_urls = list(zip(
    df_balanced['class'].values,
    df_balanced['image'].values,
))

random.shuffle(img_urls)

In [150]:
img_urls[:2]

[('skipper', 'https://bugwoodcloud.org/images/384x256/5582048.jpg'),
 ('moth', 'https://bugwoodcloud.org/images/384x256/5429059.jpg')]

The following function save a single image of an insect in it's respective folder.

In [151]:
skipped = list()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

def save_image(row):
    class_, url = row
    try:
        image_name = f"{url.split('/')[-1]}"
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            save_name = os.path.join(base_dir, class_, image_name)
            with open(save_name, 'wb') as fp:
                fp.write(res.content)
        else:
            print("Failed to download the image: STATUS CODE {}".format(res.status_code))
    except Exception as e:
        print(e)
        print("url skipped:", url)
        skipped.append(url)

Next we are going to download the images from the urls.

In [152]:
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = [executor.submit(save_image, i) for i in img_urls]
    for future in tqdm.tqdm(as_completed(futures), desc="downloading...", total=len(img_urls)):
        pass
print("Done!!")

downloading...: 100%|████████████████████████████████████████████████████████████████| 525/525 [01:57<00:00,  4.45it/s]

Done!!


After downloading these images, We want to split our dataset into `2` sets the `train` and `test`. So we are going to create the following folder structure in our `dataset` directory.


```
📁 dataset
    📁  train
      📁class_1
         - 0.jpg
         - 1.jpg
         ....
      📁  class_2
         - 0.jpg
         - 1.jpg
         ....
      .....
    📁  test
      📁  class_1
         - 0.jpg
         - 1.jpg
         ....
      📁  class_2
         - 0.jpg
         - 1.jpg
         ....
    ....
```

In [153]:
for class_ in df_filtered['class'].unique():
    train_dir = os.path.join(base_dir,'train', class_)
    test_dir = os.path.join(base_dir,'test', class_)
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

We will take the first `72` images and put them in the test folder to the respective class and the rest will be taken to the train folder of the respective class.

In [154]:
def move_to_final_destination(trg, class_:str, limit=None):
    class_dir = os.path.join(base_dir, class_)
    total = len(os.listdir(class_dir)) if limit is None else limit
    if limit is None:
        for image_name in tqdm.tqdm(os.listdir(class_dir), total=total, desc=f"moving images to the {class_} folder..."):
            src = os.path.join(class_dir, image_name)
            shutil.move(src, trg)
    else:
        for image_name in tqdm.tqdm(os.listdir(class_dir)[:limit], total=total, desc=f"moving images to the {class_} folder..."):
            src = os.path.join(class_dir, image_name)
            shutil.move(src, trg)

for class_ in df_filtered['class'].unique():
    train_dir = os.path.join(base_dir,'train', class_)
    test_dir = os.path.join(base_dir,'test', class_)
    move_to_final_destination(test_dir, class_, limit=55)
    move_to_final_destination(train_dir, class_, limit=None)


moving images to the skipper folder...: 100%|███████████████████████████████████████| 120/120 [00:00<00:00, 532.91it/s]


Next we are going to remove the `class` directories from the dataset.

In [155]:
for class_ in df_filtered['class'].unique():
    _dir = os.path.join(base_dir, class_)
    shutil.rmtree(_dir, 0o777)

driver.quit()

### Refs
1. [github.com/CrispenGari](https://github.com/CrispenGari/web-scrapping-python/blob/main/selenium/selenium.ipynb)